Examples of Loading Term Structure (futures and spot vix indexes)

In [ ]:
from src.vix_utils.download_vix_futures import VXFuturesDownloader
from src.vix_utils.location import data_dir

from pathlib import Path

import asyncio
import aiohttp

async with aiohttp.ClientSession() as session:
 
    v=VXFuturesDownloader(Path(''),session)

    await asyncio.gather(v.download_monthly_futures())


In [ ]:
Path('')

In [ ]:
data_dir()

In [ ]:
await dl.download_one_monthly_future(2024, 1)

In Jupyter, use the vix_utils asynchronous functions to the load the data
here we read the futures and spot term structure

Another way, reading the futures, then the spot structure, in sequence instead of concurrently

In [ ]:
/Users/aptperson/source/trading/vix_utils/src/vix_utils/download_vix_futures.py

In [ ]:
vix_futures_skinny_2 = await vix_utils.async_load_vix_term_structure()
vix_spot_2=await vix_utils.async_get_vix_index_histories()

In [ ]:
vix_futures_skinny_2

Another way to load the data, it won't work if you have a running event loop, 
like we do in Jupyter. If your project generally doesn't use asyncio, you can use these calls instead of the 
async_ calls to load the data.  Ignore the errors of the following cel.

In [ ]:

try:
    vix_futures_skinny_3,vix_spot_3 = vix_utils.load_vix_term_structure(),vix_utils.get_vix_index_histories()
except Exception as e:
    print("use the api functions starting with async_ to load the term structure and vix index histories")

In [ ]:
#manipulate the data to get what we want to display
vix_spot_wide=vix_utils.pivot_spot_term_structure_on_symbol(vix_spot)
vix_futures_monthly_skinny=vix_utils.select_monthly_futures(vix_futures)
vix_futures_wide=vix_utils.pivot_futures_on_monthly_tenor(vix_futures_monthly_skinny)
vix_futures_wide=vix_utils.append_continuous_maturity_one_month(vix_futures_wide)
selected_tenors=[1,1.5,2,3,4,5,6]
original_vix_futures_wide=pd.DataFrame(vix_futures_wide)
vix_futures_wide=vix_futures_wide[selected_tenors]
vix_futures_wide




In [ ]:

vix_futures_wide=vix_utils.append_continuous_maturity_one_month(vix_futures_wide)
vix_futures_wide


In [ ]:
close=vix_futures_wide.swaplevel(axis=1)["Close"]
close

In [ ]:
#plot futures
close.plot()

In [ ]:
vix_spot_wide

In [ ]:
vix_spot_wide["Close"][["VIX","VIX9D"]].plot()

In [ ]:
vix_spot_wide["Close"][["SHORTVOL"]].plot()

In [ ]:
def add_column_level(df:pd.DataFrame,var_name):
        df2=pd.DataFrame(df)
        idx=df2.columns.to_frame()
        idx.insert(0,"Variable",var_name)
        df2.columns=pd.MultiIndex.from_frame(idx)
        return df2 


Compute the basis (future price - spot price) for the varous tenors and  VIX indexes. It is really only correct
to compare the VIX index since the futures are trading on the VIX, not VIX9D etc. 

In [ ]:
vix_spot_history_closes=vix_spot_wide['Close']    
vix_basis_by_index=[add_column_level(close.sub(vix_spot_history_closes[spot_symbol],axis=0),spot_symbol+"_Basis") for spot_symbol in ["VIX9D","VIX","VIX3M","GVZ"]]

closes=add_column_level(close,"Futures")
vix_basis=pd.concat(chain([closes],vix_basis_by_index),axis=1,join="inner")
vix_basis 


In [ ]:
day_of_interest='2023-07-14'
di_spot=vix_spot_history_closes.loc[day_of_interest]
di_spot


In [ ]:
di_futures=vix_futures_wide.loc[[day_of_interest]]
di_futures

In [ ]:
di_basis=vix_basis.loc[day_of_interest]
di_basis["VIX_Basis"]

In [ ]:
cols_to_plot=[1,1.5] +  list(range(2,5))
df_day_of_interest_to_plot=di_futures.swaplevel(axis=1)[['Close',"Tenor_Days"]]

#.swaplevel(axis=1)[cols_to_plot].swaplevel()
dx=df_day_of_interest_to_plot.T
df_day_of_interest_to_plot


In [ ]:
df_day_of_interest_to_plot.plot(x="Tenor_Days", y="Close", kind = 'scatter', use_index=True)